In [ ]:
from pathlib import Path
import uchronia.data_set as uds
from uchronia.sample_data import sample_data_dir
import os

## -----------------------------------------------------------------------------
s = sample_data_dir()
data_library_file = os.path.join(s, 'time_series_library.yaml')
assert Path(data_library_file).exists()

In [ ]:
## -----------------------------------------------------------------------------
data_library = uds.get_ensemble_dataset(data_library_file, '')
data_ids = data_library.get_dataset_ids()
data_ids

## -----------------------------------------------------------------------------
single_obs = data_library.get_dataset("var1_obs")
single_obs
asXts = uds.as_xarray
asXts(single_obs).plot()

import uchronia.time_series as uts
## -----------------------------------------------------------------------------
stationIds =  data_library.sub_identifiers("var1_obs_collection")
print(stationIds)
multiObs = data_library.get_dataset("var1_obs_collection")

multiObs

multiObs = asXts(multiObs)
multiObs.head()
multiObs.plot()

## -----------------------------------------------------------------------------
uds.datasets_summaries(data_library)

## -----------------------------------------------------------------------------
fcasts = data_library.get_dataset("var2_fcast_ens")
fcasts
# TODO: currently conversion ot xts is the default. Review:
firstFcast = fcasts.get_item(1)
firstFcast.head()
firstFcast.plot()

# -

# Ensemble SWIFT model runs

## About this document

## Elaboration

In [ ]:
import pandas as pd
import numpy as np
import xarray as xr

In [ ]:
## -----------------------------------------------------------------------------
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline

the package `uchronia` includes facilities to access time series from a "library", akin to what you would do to manage books.

In [ ]:
import uchronia.sample_data as usd

In [ ]:
doc_data_path = usd.sample_data_dir()
data_path = os.path.join(doc_data_path, 'UpperMurray')
data_path

In [ ]:
data_library = usd.sample_time_series_library('upper murray')

In [ ]:
data_library

In [ ]:
data_ids = data_library.get_dataset_ids()

In [ ]:
data_ids

In [ ]:
data_library.datasets_summaries()
# 'rain_obs'
# 'pet_obs'

In [ ]:
x = data_library.get_dataset('rain_obs')

In [ ]:
b = uts.as_xarray(x);

In [ ]:
def _is_positive(x): return x > 0
# b[_is_negative(b)]

In [ ]:
b.where(lambda arr: _is_positive(arr), np.nan).plot()

In [ ]:
from cinterop.cffi.marshal import as_timestamp

First, before demonstrating ensemble forecasting simulations, let's demonstrate how we can get a snapshot of the model states at a point in time and restore it later on, hot-starting further simulation.

In [ ]:
ens_rain = data_library.get_dataset('rain_fcast_ens')

In [ ]:
one_forecast = ens_rain.get_item(1)

In [ ]:
one_forecast

In [ ]:
q = one_forecast.quantile([0.05, .25, .5, .75, 0.95], 'ensemble')

In [ ]:
fig, ax = plt.subplots(figsize=(16,9))
ax.fill_between(q.time.values, q.sel(quantile=0.05), q.sel(quantile=0.95), alpha=0.3, label='Perc. 50-95')
ax.fill_between(q.time.values, q.sel(quantile=0.25), q.sel(quantile=.75), alpha=0.5, label='Perc. 25-75')
ax._get_lines.get_next_color()  # Hack to get different line
ax.plot(q.time.values, q.sel(quantile=.5), linewidth=2, label='Median')
ax.legend()
plt.show()